# DOE Data to Bronze Layer

This notebook processes DOE (Department of Energy) CSV and Excel files to Delta tables.
Data includes energy consumption, electricity generation, power plant statistics, and energy sector data.

Features:
- Multiple encoding support for robust file processing
- Handles both CSV and Excel files
- Skip bad lines automatically
- Comprehensive error handling and reporting
- Empty row filtering and data cleaning
- Automatic data type conversion

In [ ]:
# Initialize Spark session with Delta Lake
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, when, regexp_replace
from pyspark.sql.types import StringType, IntegerType, DoubleType
from delta import configure_spark_with_delta_pip
import re
import json
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

builder = SparkSession.builder \
    .appName("DOE-Bronze") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print("Spark session initialized successfully")
print(f"Spark version: {spark.version}")

In [ ]:
# Define paths and helper functions
doe_data_path = "../DOE"
bronze_layer_path = "../final-spark-bronze/bronze_doe"

os.makedirs(bronze_layer_path, exist_ok=True)

def clean_column_name(col_name):
    """Clean column names for Delta compatibility"""
    if not col_name or col_name.strip() == "":
        return "unnamed_column"
    
    cleaned = str(col_name).strip().replace('"', '').replace("'", '')
    cleaned = re.sub(r'[^a-zA-Z0-9_]', '_', cleaned)
    cleaned = re.sub(r'_+', '_', cleaned).strip('_')
    
    if not cleaned:
        cleaned = "unnamed_column"
    elif cleaned[0].isdigit():
        cleaned = f"col_{cleaned}"
    
    return cleaned

def clean_table_name(filename):
    """Generate clean table name from filename"""
    table_name = filename.replace('.csv', '').replace('.xlsx', '').replace('.xls', '')
    table_name = clean_column_name(table_name)
    
    # Add prefix if starts with number
    if table_name[0].isdigit():
        table_name = f"doe_{table_name}"
    else:
        table_name = f"doe_{table_name}"
    
    if len(table_name) > 100:
        table_name = table_name[:100].rstrip('_')
    
    return table_name

def detect_csv_format(file_path):
    """Detect CSV encoding and separator"""
    encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
    separators = [',', ';', '\t']
    
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                first_line = f.readline().strip()
            
            # Detect best separator
            best_sep = ','
            max_splits = 0
            
            for sep in separators:
                splits = len(first_line.split(sep))
                if splits > max_splits:
                    max_splits = splits
                    best_sep = sep
            
            return encoding, best_sep
            
        except UnicodeDecodeError:
            continue
    
    return 'latin1', ','

def read_csv_with_pandas(file_path):
    """Read CSV using pandas with robust handling"""
    try:
        encoding, separator = detect_csv_format(file_path)
        print(f"    Detected encoding: {encoding}, separator: '{separator}'")
        
        df_pandas = pd.read_csv(
            file_path,
            encoding=encoding,
            sep=separator,
            na_values=['', 'N/A', 'n/a', 'NULL', 'null', '-'],
            keep_default_na=True,
            dtype=str,
            on_bad_lines='skip',
            skipinitialspace=True
        )
        
        # Clean up the DataFrame
        original_shape = df_pandas.shape
        df_pandas = df_pandas.dropna(how='all')
        df_pandas = df_pandas[~df_pandas.apply(lambda x: x.str.strip().eq('').all(), axis=1)]
        
        if df_pandas.empty:
            return None, False, "No valid data rows found after cleaning"
        
        print(f"    Cleaned data: {original_shape} -> {df_pandas.shape}")
        
        return df_pandas, True, None
        
    except Exception as e:
        return None, False, f"CSV read error: {str(e)}"

def read_excel_with_pandas(file_path):
    """Read Excel using pandas"""
    try:
        # Get all sheet names
        excel_file = pd.ExcelFile(file_path)
        print(f"    Found {len(excel_file.sheet_names)} sheets: {excel_file.sheet_names}")
        
        # Read first sheet (or main data sheet)
        sheet_name = excel_file.sheet_names[0]
        df_pandas = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            na_values=['', 'N/A', 'n/a', 'NULL', 'null', '-'],
            keep_default_na=True
        )
        
        # Convert all columns to string to maintain consistency
        df_pandas = df_pandas.astype(str)
        
        # Clean up the DataFrame
        original_shape = df_pandas.shape
        df_pandas = df_pandas.dropna(how='all')
        df_pandas = df_pandas[~df_pandas.apply(lambda x: x.str.strip().eq('').all(), axis=1)]
        
        if df_pandas.empty:
            return None, False, "No valid data rows found after cleaning"
        
        print(f"    Cleaned data: {original_shape} -> {df_pandas.shape}")
        print(f"    Using sheet: {sheet_name}")
        
        return df_pandas, True, None
        
    except Exception as e:
        return None, False, f"Excel read error: {str(e)}"

def clean_and_convert_dataframe(df_pandas):
    """Clean column names and handle duplicates"""
    # Clean column names
    df_pandas.columns = [clean_column_name(col) for col in df_pandas.columns]
    
    # Handle duplicate column names
    seen_columns = {}
    new_columns = []
    for col in df_pandas.columns:
        if col in seen_columns:
            seen_columns[col] += 1
            new_columns.append(f"{col}_{seen_columns[col]}")
        else:
            seen_columns[col] = 0
            new_columns.append(col)
    
    df_pandas.columns = new_columns
    
    # Convert to Spark DataFrame
    df_spark = spark.createDataFrame(df_pandas)
    
    return df_spark

print("Helper functions defined")
print(f"Source path: {doe_data_path}")
print(f"Target path: {bronze_layer_path}")

In [ ]:
# Get list of DOE files
doe_files = [f for f in os.listdir(doe_data_path) if f.endswith(('.csv', '.xlsx', '.xls'))]
doe_files.sort()

print(f"Found {len(doe_files)} DOE files to process")
print("Files to process:")
for i, file in enumerate(doe_files, 1):
    file_ext = file.split('.')[-1].upper()
    file_size = os.path.getsize(os.path.join(doe_data_path, file)) / 1024
    print(f"  {i:2d}. {file} ({file_ext}, {file_size:.1f} KB)")

In [ ]:
# Process all DOE files
processed_tables = []
failed_files = []
processing_details = []

print(f"Processing all {len(doe_files)} files...")

for i, filename in enumerate(doe_files, 1):
    print(f"\nProcessing {i}/{len(doe_files)}: {filename}")
    
    table_name = clean_table_name(filename)
    file_path = os.path.join(doe_data_path, filename)
    file_ext = filename.split('.')[-1].lower()
    
    try:
        # Read file based on extension
        if file_ext == 'csv':
            df_pandas, parse_success, parse_error = read_csv_with_pandas(file_path)
        elif file_ext in ['xlsx', 'xls']:
            df_pandas, parse_success, parse_error = read_excel_with_pandas(file_path)
        else:
            parse_success = False
            parse_error = f"Unsupported file type: {file_ext}"
            df_pandas = None
        
        if not parse_success:
            print(f"  Failed to parse: {parse_error}")
            failed_files.append((filename, parse_error))
            processing_details.append({
                'filename': filename,
                'error': parse_error,
                'status': 'failed'
            })
            continue
        
        # Convert to Spark DataFrame
        df_spark = clean_and_convert_dataframe(df_pandas)
        
        row_count = df_spark.count()
        col_count = len(df_spark.columns)
        
        print(f"  Processed: {row_count} rows, {col_count} columns")
        
        if row_count == 0:
            print(f"  Skipped: No data rows")
            failed_files.append((filename, "No data rows found"))
            processing_details.append({
                'filename': filename,
                'error': "No data rows found",
                'status': 'failed'
            })
            continue
        
        # Save to Delta
        delta_path = os.path.join(bronze_layer_path, table_name)
        
        df_spark.write.format("delta") \
          .mode("overwrite") \
          .option("delta.columnMapping.mode", "name") \
          .option("delta.minReaderVersion", "2") \
          .option("delta.minWriterVersion", "5") \
          .save(delta_path)
        
        # Verify
        df_check = spark.read.format("delta").load(delta_path)
        verify_count = df_check.count()
        
        print(f"  Saved and verified: {verify_count} rows")
        processed_tables.append(table_name)
        
        processing_details.append({
            'filename': filename,
            'table_name': table_name,
            'file_type': file_ext,
            'rows': verify_count,
            'columns': col_count,
            'status': 'success'
        })
        
    except Exception as e:
        error_msg = f"Processing error: {str(e)}"
        print(f"  {error_msg}")
        failed_files.append((filename, error_msg))
        processing_details.append({
            'filename': filename,
            'error': error_msg,
            'status': 'failed'
        })

print(f"\nProcessing complete!")
print(f"Successfully processed: {len(processed_tables)} tables")
print(f"Failed: {len(failed_files)} files")
if len(processed_tables) + len(failed_files) > 0:
    success_rate = len(processed_tables)/(len(processed_tables)+len(failed_files))*100
    print(f"Success rate: {success_rate:.1f}%")

In [ ]:
# Display results summary
print("\nPROCESSING SUMMARY")
print("=" * 60)

if failed_files:
    print(f"\nFailed files ({len(failed_files)}):")
    for i, (filename, error) in enumerate(failed_files, 1):
        print(f"  {i:2d}. {filename}")
        print(f"      Error: {error[:100]}..." if len(error) > 100 else f"      Error: {error}")

print(f"\nSuccessfully created Delta tables ({len(processed_tables)}):")
successful_details = [d for d in processing_details if d['status'] == 'success']
for i, detail in enumerate(successful_details, 1):
    print(f"  {i:2d}. {detail['table_name']}")
    print(f"      Source: {detail['filename']} ({detail['file_type'].upper()})")
    print(f"      Data: {detail['rows']:,} rows, {detail['columns']} columns")

# Statistics
if successful_details:
    total_rows = sum(d['rows'] for d in successful_details)
    avg_columns = sum(d['columns'] for d in successful_details) / len(successful_details)
    
    # File type breakdown
    file_types = {}
    for detail in successful_details:
        ft = detail['file_type'].upper()
        file_types[ft] = file_types.get(ft, 0) + 1
    
    print(f"\nStatistics:")
    print(f"  Total rows processed: {total_rows:,}")
    print(f"  Average columns per table: {avg_columns:.1f}")
    print(f"  File type breakdown: {dict(file_types)}")
    if successful_details:
        largest = max(successful_details, key=lambda x: x['rows'])
        print(f"  Largest table: {largest['table_name']} ({largest['rows']:,} rows)")

In [ ]:
# Save processing report
report = {
    'timestamp': datetime.now().isoformat(),
    'data_source': 'DOE',
    'total_files': len(doe_files),
    'successful_tables': len(processed_tables),
    'failed_files': len(failed_files),
    'success_rate': len(processed_tables)/(len(processed_tables)+len(failed_files))*100 if (len(processed_tables)+len(failed_files)) > 0 else 0,
    'processing_details': processing_details,
    'processed_tables': processed_tables,
    'failed_files': [{'filename': f, 'error': e} for f, e in failed_files]
}

report_path = os.path.join(bronze_layer_path, 'processing_report.json')
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f"\nProcessing report saved to: {report_path}")

In [ ]:
# Example table validation
if processed_tables:
    print("\nEXAMPLE TABLE VALIDATION")
    print("=" * 50)
    
    example_table = processed_tables[0]
    example_path = os.path.join(bronze_layer_path, example_table)
    
    print(f"Validating: {example_table}")
    
    df_example = spark.read.format("delta").load(example_path)
    
    print(f"\nTable statistics:")
    print(f"  Rows: {df_example.count():,}")
    print(f"  Columns: {len(df_example.columns)}")
    
    print(f"\nColumn names (first 5):")
    for col_name in df_example.columns[:5]:
        print(f"  - {col_name}")
    
    print(f"\nSample data:")
    df_example.show(3, truncate=True)
    
    print(f"\nData quality check:")
    total_rows = df_example.count()
    for col_name in df_example.columns[:3]:
        null_count = df_example.filter(col(col_name).isNull()).count()
        null_pct = (null_count / total_rows) * 100 if total_rows > 0 else 0
        print(f"  {col_name}: {null_count:,} nulls ({null_pct:.1f}%)")

In [ ]:
# Stop Spark session
spark.stop()
print("Spark session stopped")
print("\nProcessing complete. Check the bronze_doe directory for Delta tables.")